I will be working with fault code data and vehicle onboard diagnostic data to try and predict an upcoming full derate. These are indicated by an SPN 5246.

In [1]:
import pandas as pd

In [2]:
parse_dates=['EventTimeStamp']

In [3]:
faults = pd.read_csv('data/J1939Faults.csv', low_memory=False, parse_dates=['EventTimeStamp'])
service_fault = pd.read_excel('data/ServiceFaultCodes.xlsx')
vehicle_Diagnostics = pd.read_csv('data/VehicleDiagnosticOnboardData.csv')

/Users/alisoncordoba/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


Basic EDA to check how the data looks like:

In [4]:
#get rid of faultValue and actionDescription since they haven't been filled in. 
#FaultId = RecordID

faults

,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,faultValue,EquipmentID,MCTNumber,Latitude,Longitude,LocationTimeStamp
0,1,990349,2015-02-21 10:47:13,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,0,111,17,True,2,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:34:25.000
1,2,990360,2015-02-21 11:34:34,NaN,NaN,unknown,unknown,unknown,unknown,11,629,12,True,127,NaN,1439,105354361,38.857638,-84.626851,2015-02-21 11:35:10.000
2,3,990364,2015-02-21 11:35:31,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,False,127,NaN,1369,105336226,41.421250,-87.767361,2015-02-21 11:35:26.000
3,4,990370,2015-02-21 11:35:33,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,11,1807,2,True,127,NaN,1369,105336226,41.421018,-87.767361,2015-02-21 11:36:08.000
4,5,990416,2015-02-21 11:39:41,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,0,4364,17,False,2,NaN,1674,105427130,38.416481,-89.442638,2015-02-21 11:39:37.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,123904424,2020-03-06 14:00:26,Low (Severity Low) Catalyst Tank Level,NaN,04384413*22383729*082218154102*60701732*G1*BGT*,80156139,6X1u17D1500000000,CMMNS,0,1761,17,False,3,NaN,2282,105439740,37.094768,-85.897407,2020-03-06 14:00:21.000
1187331,1248455,123905139,2020-03-06 14:04:23,Condition Exists Engine Protection Torque Derate,NaN,04358814*06099720*030816202706*09400153*G1*BDR*,79932020,6X1u13D1500000000,CMMNS,0,1569,31,True,5,NaN,1994,105354084,34.390740,-79.461805,2020-03-06 14:04:59.000
1187332,1248456,123905996,2020-03-06 14:13:38,Abnormal Rate of Change Aftertreatment 1 Intak...,NaN,05317106*05100987*050719120655*09401585*G1*BDR*,79880653,6X1u13D1500000000,CMMNS,0,3216,10,True,1,NaN,1850,105336308,34.430370,-84.920509,2020-03-06 14:14:14.000
1187333,1248457,123906113,2020-03-06 14:14:13,Low (Severity Medium) Engine Coolant Level,NaN,04384413*22544852*090619141107*60701756*G1*BGT*,NaN,NaN,NaN,0,111,18,True,8,NaN,2377,108605700,35.030925,-85.321527,2020-03-06 14:14:49.000


In [5]:
faults.shape

(1187335, 20)

In [6]:
faults.isna().sum()

RecordID                       0
ESS_Id                         0
EventTimeStamp                 0
eventDescription           60845
actionDescription        1187335
ecuSoftwareVersion        296050
ecuSerialNumber           343017
ecuModel                   64758
ecuMake                    64758
ecuSource                      0
spn                            0
fmi                            0
active                         0
activeTransitionCount          0
faultValue               1187335
EquipmentID                    0
MCTNumber                      0
Latitude                       0
Longitude                      0
LocationTimeStamp              0
dtype: int64

In [7]:
faults.describe()

,RecordID,ESS_Id,actionDescription,ecuSource,spn,fmi,activeTransitionCount,faultValue,MCTNumber,Latitude,Longitude
count,1.187335e+06,1.187335e+06,0.0,1.187335e+06,1.187335e+06,1.187335e+06,1.187335e+06,0.0,1.187335e+06,1.187335e+06,1.187335e+06
mean,6.116411e+05,2.509278e+07,NaN,2.249565e+01,2.749409e+03,1.067145e+01,7.721882e+01,NaN,1.054267e+08,3.652602e+01,-8.406577e+01
std,3.574954e+05,2.935562e+07,NaN,2.342841e+01,2.663703e+04,7.095709e+00,5.908779e+01,NaN,3.085221e+05,2.937647e+00,4.578389e+00
min,1.000000e+00,9.903490e+05,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,1.053013e+08,0.000000e+00,-1.233710e+02
25%,3.013395e+05,6.148916e+06,NaN,0.000000e+00,1.110000e+02,3.000000e+00,1.000000e+00,NaN,1.053561e+08,3.520194e+01,-8.643481e+01
50%,6.081460e+05,1.212963e+07,NaN,1.100000e+01,6.390000e+02,9.000000e+00,1.260000e+02,NaN,1.054104e+08,3.615227e+01,-8.462611e+01
75%,9.145655e+05,3.256915e+07,NaN,4.900000e+01,9.290000e+02,1.700000e+01,1.260000e+02,NaN,1.054273e+08,3.855440e+01,-8.150120e+01
max,1.248458e+06,1.239061e+08,NaN,6.100000e+01,5.242870e+05,3.100000e+01,1.270000e+02,NaN,1.086168e+08,4.810009e+01,0.000000e+00


In [8]:
service_fault

,Published in CES 14602,Cummins Fault Code,Revision,PID,SID,MID,J1587 FMI,SPN,J1939 FMI,J2012 Pcode,Lamp Color,Lamp Device,Cummins Description,Algorithm Description
0,Y,111,167,Not Mapped,254,0,12,629,12,P0606,Red,Stop / Shutdown,Engine Control Module Critical Internal Failur...,Error internal to the ECM related to memory ha...
1,Y,112,167,Not Mapped,20,128,7,635,7,Not Mapped,Red,Stop / Shutdown,Engine Timing Actuator Driver Circuit - Mechan...,Mechanical failure in the engine timing actuat...
2,Y,113,167,Not Mapped,20,128,3,635,3,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,High signal voltage detected at the engine tim...
3,Y,114,167,Not Mapped,20,128,4,635,4,Not Mapped,Amber,Warning,Engine Timing Actuator Driver Circuit - Voltag...,Low voltage detected at the engine timing actu...
4,Y,115,167,190,Not Mapped,Not Mapped,2,612,2,P0008,Red,Stop / Shutdown,Engine Magnetic Speed/Position Lost Both of Tw...,The ECM has detected that the primary and back...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7119,Y,9996,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN
7120,Y,9997,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN
7121,Y,9998,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN
7122,Y,9999,167,Not Mapped,155,0,11,524286,31,Not Mapped,Amber,Warning,Reserved for temporary use - Condition Exists,NaN


In [9]:
service_fault.shape

(7124, 14)

In [10]:
service_fault.isna().sum()

Published in CES 14602       0
Cummins Fault Code           0
Revision                     0
PID                          0
SID                          0
MID                          0
J1587 FMI                    0
SPN                          0
J1939 FMI                    0
J2012 Pcode                  0
Lamp Color                   0
Lamp Device                  0
Cummins Description          0
Algorithm Description     5119
dtype: int64

In [11]:
service_fault.describe()

,Cummins Fault Code,Revision,J1587 FMI,SPN,J1939 FMI
count,7124.000000,7124.0,7124.000000,7124.000000,7124.000000
mean,5134.143880,167.0,6.376193,195036.763616,15.626193
std,2860.623551,0.0,4.554717,250969.003813,11.841491
min,111.000000,167.0,0.000000,22.000000,0.000000
25%,2691.750000,167.0,2.000000,1277.000000,4.000000
50%,5191.500000,167.0,5.000000,5020.000000,14.000000
75%,7589.500000,167.0,11.000000,520941.250000,31.000000
max,12345.000000,167.0,14.000000,524287.000000,31.000000


In [12]:
vehicle_Diagnostics

,Id,Name,Value,FaultId
0,1,IgnStatus,False,1
1,2,EngineOilPressure,0,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0,1
4,5,EngineLoad,11,1
...,...,...,...,...
12821621,12864020,EngineCoolantTemperature,181.4,1248457
12821622,12864021,ParkingBrake,False,1248457
12821623,12864022,SwitchedBatteryVoltage,14.1,1248457
12821624,12864023,DistanceLtd,28606.65625,1248457


In [13]:
vehicle_Diagnostics.shape

(12821626, 4)

In [14]:
vehicle_Diagnostics.isna().sum()

Id         0
Name       0
Value      0
FaultId    0
dtype: int64

In [15]:
vehicle_Diagnostics.describe()

,Id,FaultId
count,1.282163e+07,1.282163e+07
mean,6.423667e+06,6.197873e+05
std,3.711136e+06,3.575742e+05
min,1.000000e+00,1.000000e+00
25%,3.209463e+06,3.084730e+05
50%,6.421294e+06,6.214040e+05
75%,9.633220e+06,9.242960e+05
max,1.286402e+07,1.248458e+06
